In [1]:
from PLANBERT.Model import PLANBERT
import pandas as pd
# Load the training, validation, and testing set.
master_csv = pd.read_csv('example/example_master.csv')
train_len, tv_len = 153681, 153681+31661
train_csv, valid_csv, test_csv = master_csv.iloc[:train_len], master_csv.iloc[train_len:tv_len], master_csv.iloc[tv_len:]

Using TensorFlow backend.


In [2]:
# Train a PLAN-BERT with training set and validation set without checkpoint.
planbert = PLANBERT(num_times=6, num_items=10000, feat_dims=[5000, 1000], cuda_num=0) # [ Number of time slots, Number of items, [Number of features], ID of GPU]

In [3]:
test_csv_history = test_csv[test_csv['t'] < 4]
test_csv_future = test_csv[test_csv['t'] >= 4]#.sample(frac=0.1)

In [4]:
predict = planbert.predict(
    test_csv_history, 
    test_csv_future, 
    mode='time', 
    pred_time_slices=4, 
    items_per_slice=20
)
predict

100%|██████████| 48/48 [00:02<00:00, 17.10it/s]


,user,t,item,prob
0,0,2,222,0.000852
1,0,2,5810,0.000851
2,0,2,799,0.000770
3,0,2,7891,0.000760
4,0,2,6717,0.000734
...,...,...,...,...
3835,894,5,1131,0.000559
3836,894,5,4416,0.000548
3837,894,5,7108,0.000548
3838,894,5,6957,0.000535
